# 00 - PostgreSQL Through Jupyter


This is an optional notebook for loading data through PostgreSQL. <br>
The data should be in the form of a .csv file.

## 1. Import Packages <a name="import"></a>

In [229]:
import csv
import pandas as pd

import pandas.io.sql as pd_sql

from collections import defaultdict

import os

import psycopg2 as pg
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2 import OperationalError, errorcodes, errors

from tqdm import tqdm

import pprint as pp

## Table of Contents <a name="table"></a>
1. [Import Packages](#import)
2. [Import Data](#import_data)
3. [CREATE TABLE Statement](#create)
4. [Connecting to PostgreSQL](#connect)
5. [INSERT TABLE Statement](#insert)
6. [Inserting Data into PostgreSQL](#insert_data)
    1. [Clean Data](#clean)
    2. [Messy Data](#messy)
7. [Querying Data](#query)
    1. [Cursor Method](#cursor)
    2. [Pandas Method](#pandas)
8. [Closing the Connection](#close)


## 2. Import Data <a name="import_data"></a>

We import a sample of the .csv file to check what data types it has.

In [244]:
#get the file path of the data
file_dir = os.path.abspath('.')
data_folder = 'data'
csv_file = 'ks-projects-201801.csv.gz'

path = os.path.join(file_dir, data_folder, csv_file)

#load only a portion of the csv file, specifically 10 rows with nrows = 10
ks2018_sample = pd.read_csv(path, low_memory = False, nrows = 10)
ks2018_sample.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [22]:
ks2018_dtypes = pd.Series(ks2018_sample.dtypes)
ks2018_dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers               int64
country              object
usd pledged         float64
usd_pledged_real    float64
usd_goal_real       float64
dtype: object

Return to [Table of Contents](#table)

## 3. CREATE TABLE Statement <a name="create"></a>

In [150]:
def create_table_statement(df, table_name):
    """
        Helper function to make "CREATE TABLE" statement for psql
        
        :param df: a pandas DateFrame object
        :param table_name: name of new table in psql as string
        :return create_table: "CREATE TABLE" statement for psql as string
    """
    
    col_names = df.dtypes.index.values
    dtypes = df.dtypes.values

    #every name in psql is lowercase by default
    table_name = table_name.lower()

    #statement for creating the table in psql
    create_table = "CREATE TABLE IF NOT EXISTS %s (\n" %table_name  

    for idx, col_name in enumerate(col_names):
        #get the datatype for the current column
        dtype = dtypes[idx]

        #lowercase and remove spaces from column names
        col_name = col_name.lower().replace(' ', '_')
        #simply formatting so it prints more cleanly
        create_table += "\t"

        #assign column types and default values of null
        if dtype == 'int64':
            create_table = create_table + col_name + " INT DEFAULT NULL"

        elif dtype == 'object':
            create_table = create_table + col_name + " VARCHAR DEFAULT NULL"

        elif dtype == 'float64':
            create_table = create_table + col_name + " DECIMAL DEFAULT NULL"

        #add a comma and start next line on 
        #another line for cleaner print format
        #if this is not the last line
        if idx != len(col_names) - 1:
            create_table += ", \n"

    #newline for cleaner print format and ; for execution
    create_table += "\n);"

    print(create_table)
    
    return create_table

In [151]:
create_table = create_table_statement(ks2018_sample, 'ks2018')

CREATE TABLE IF NOT EXISTS ks2018 (
	id INT DEFAULT NULL, 
	name VARCHAR DEFAULT NULL, 
	category VARCHAR DEFAULT NULL, 
	main_category VARCHAR DEFAULT NULL, 
	currency VARCHAR DEFAULT NULL, 
	deadline VARCHAR DEFAULT NULL, 
	goal DECIMAL DEFAULT NULL, 
	launched VARCHAR DEFAULT NULL, 
	pledged DECIMAL DEFAULT NULL, 
	state VARCHAR DEFAULT NULL, 
	backers INT DEFAULT NULL, 
	country VARCHAR DEFAULT NULL, 
	usd_pledged DECIMAL DEFAULT NULL, 
	usd_pledged_real DECIMAL DEFAULT NULL, 
	usd_goal_real DECIMAL DEFAULT NULL
);


Return to [Table of Contents](#table)

## 4. Connecting to Postgresql <a name="connect"></a>

If you experience troubles using the connection_args below, make sure:
- PostgreSQL server has been started: <br>
`sudo service postgresql start` <br>

- The PostgreSQL server is listening: <br>
`sudo su - postgres`<br>
`vim /etc/postgresql/10/main/postgresql.conf` <br>
`# listen_addresses='localhost'` needs to be uncommented <br>
`sudo service postgresql restart` after exiting out of postgres super account <br>

- A password exists for the user account if a password if requested: <br>
`sudo su - postgres`<br>
`psql` <br>
`ALTER USER kchiv PASSWORD 'kchiv';` while using the postgres super account in psql <br>
`sudo service postgresql restart` after exiting out of postgres super account <br>

In [53]:
connection_args = {'host': 'localhost', #for ec2, use ip address instead
                   'user':'kchiv', #change this to your username
                   'password':'kchiv', #change this to your password, leave out if unnecessary
                   'port': 5432}

In [231]:
connection = pg.connect(**connection_args)

#if you want to autocommit instead, comment out if not
connection.autocommit = True

cursor = connection.cursor()

#if you want to autocommit instead, comment out if not
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) 

PostgreSQL does not support "CREATE DATABASE <database> IF NOT EXISTS;"

In [75]:
#do this if you need to create the database
#everything in psql is lowercase by default
db_name = 'kickstarter'

try:
    cursor.execute("CREATE DATABASE %s  ;" %db_name)
except Exception as error:
    print('Duplicate Database: %s database already exists' %db_name)

Duplicate Database: kickstarter database already exists


In [77]:
#change this value to true if you want to drop the database
drop_database = False

if drop_database:
    #run the follow code if you need to drop the database
    cursor.execute("DROP DATABASE %s  ;" %db_name) 

Connect to the database by restarting the connection with the name of the database provided.

In [62]:
connection.close()

connection_args['dbname'] = 'kickstarter'
connection = pg.connect(**connection_args)

cursor = connection.cursor()

#if you want to autocommit instead, comment out if not
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [159]:
#create table in psql
cursor.execute(create_table)

In [158]:
#change this value to true if you want to drop the table
drop_table = False

if drop_table:
    #run the follow code if you need to drop the table
    cursor.execute("DROP Table IF EXISTS %s  ;" %table_name) 

Return to [Table of Contents](#table)

## 5. INSERT TABLE Statement <a name="insert"></a>

In [152]:
def insert_table_statement(df, table_name):
    """
        Function to make "INSERT INTO" statement for psql
        
        :param df: a pandas DateFrame object
        :param table_name: name of new table in psql as string
        :return insert_statement: "INSERT INTO" statement for psql as string
    """
    
    col_names = df.columns
    
    #statement for inserting values into table
    insert_statement = "INSERT INTO %s (" %table_name
    values = " VALUES ("
    insert_statement

    for idx, col_name in enumerate(col_names):

        #lowercase and remove spaces from column names
        #as we did before with the table
        col_name = col_name.lower().replace(' ', '_')

        #adding columns to insert statement
        #also formatting after every 4th column for cleaner print format
        if idx % 4 == 0 and idx != 0:
            insert_statement = insert_statement + '\n\t\t\t' + col_name
        else:
            insert_statement = insert_statement + col_name 

        #append a %s to values for every col_name
        #this is for substituting the values in later
        values += "%s"

        #exit loop if this the last column
        if idx == len(col_names) - 1:
            break

        insert_statement += ", "
        values += ", "

    insert_statement += ")"
    values += ");"

    insert_statement = insert_statement + '\n' + values

    print(insert_statement)
    
    return insert_statement

Return to [Table of Contents](#table)

## 6. Inserting Data into PostgreSQL <a name="insert_data"></a>

You can insert data directly into PostgreSQL via Jupyter as long as these conditions are met:
- table has already been created in PostgreSQL
- Jupyter notebook is connected to PostgreSQL via psycopg2 

### 6A. Clean Data <a name="clean"></a>

You can insert data directly into PostgreSQL via Jupyter as long as these conditions are met:
- .csv file of data exists
- data is clean i.e. data has the correct types

In [ ]:
#get the file path of the data
file_dir = os.path.abspath('.')
data_folder = 'data'
csv_file = 'ks-projects-201801.csv'

path = os.path.join(file_dir, data_folder, csv_file)

table_name = 'ks2018'

#COPY statement in psql as string
copy_statement = "COPY " + table_name + " FROM " + path + " DELIMITER ',' CSV HEADER;"

#copy data to psql table
cursor.execute(copy_statement)

#save data to psql table
connection.commit()

Return to [Table of Contents](#table)

### 6B. Messy Data <a name="messy"></a>

Data is usually messy and you want to use PostgreSQL to examine the data might be too large for pandas or any other method. <br>
The function below inserts data one row at a time from a .csv file. <br>
Rows that cannot be inserted from the .csv file are saved to an error file named error.csv. <br>
Fair warning, it could take a while to run.

In [164]:
def insert_into_table(connection, insert_statement, file_dir, data_folder, csv_file, encoding):

    """
        Function to insert data into table in psql when data is not clean
        This assumes the table already exists in psql
        Could take a while to run, depending on the size of the .csv file
        
        :param connection: connection from psycopg2
        :param insert_statement: 'INSERT INTO' statement produced by insert_statement()
        :param file_dir: absolute path of working directory
        :param data_folder: name of folder to fetch and save data
        :param csv_file: name of .csv file
        :param encoding: .csv file encoding
    """
    
    path = os.path.join(file_dir, data_folder, csv_file)
    
    cursor = connection.cursor()
    
    #keep track of the number of correct inserts
    num_correct = 0

    #may have to change encoding depending on file
    with open(path, mode = 'r', encoding = encoding) as csv_file: 
        reader = csv.reader(csv_file, delimiter = ',')

        #skip the headers i.e. colummn names
        #save the column names info for later
        col_names = next(reader)

        #lowercase and replace spaces with '_'
        #to match psql statements
        col_names = [col_name.lower().replace(' ', '_') 
                         for col_name in col_names]

        #iterating through the rows of the .csv file
        #wrap iterable with tqdm to see progress
        for idx, row in tqdm(enumerate(reader)):
            try:
                cursor.execute(insert_statement, tuple(row))
                num_correct += 1

            except Exception as error:
                error_path = os.path.join(file_dir, data_folder, 'errors.csv')
                with open(error_path, mode = 'a') as error_file:
                    #we add one to idx since .csv files start at index one
                    error_file.write(str(idx+1))
            
            #occasionally save what was inserted
            #into the psql table after ever 1000th row 
            if idx % 1000 and idx != 0:
                connection.commit()
        
        #final save to table in psql
        connection.commit()
        cursor.close()
    
    print('Number of Successful Row Inserts: %s' %num_correct)

In [165]:
#change to true if you want to remake the table
drop = False

if drop:
    #drop table in case of mistakes, not necessary
    cursor.execute("DROP Table IF EXISTS %s  ;" %table_name) 

    #make new table if table is dropped
    cursor.execute(create_table)

#change this to true if you want to insert
insert = False

if insert:
    insert_statement = insert_table_statement(ks2018_sample, 'ks2018')

    file_dir = os.path.abspath('.')
    data_folder = 'data'
    csv_file = 'ks-projects-201801.csv'

    insert_into_table(connection, insert_statement, 
                      file_dir, data_folder, csv_file, encoding = 'utf8')

18it [00:00, 170.55it/s]

INSERT INTO ks2018 (id, name, category, main_category, 
			currency, deadline, goal, launched, 
			pledged, state, backers, country, 
			usd_pledged, usd_pledged_real, usd_goal_real)
 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);


378661it [53:29, 117.96it/s]

Number of Successful Row Inserts: 374864


Check for errors after all the data has been inserted.

In [168]:
if insert:
    error_path = os.path.join(file_dir, data_folder, 'errors.csv')

    with open(error_path, mode = 'r') as error_file:
        reader = csv.reader(error_file, delimiter = ',')

        error_indices = [row for row in reader]

    print('Number of Rows that Errored Out: %s' %len(error_indices))

Number of Rows that Errored Out: 1


Return to [Table of Contents](#table)

## 7. Querying Data <a name="query"></a>

### 7A. Cursor Method <a name="cursor"></a>

The cursor method is useful for when you have too much data stored in the table in PostgreSQL. <br>
However, it can be rather difficult to look at the data.

In [220]:
query = "SELECT main_category, category, count(id) FROM ks2018 \
         GROUP BY main_category, category \
         ORDER BY main_category, COUNT(id) DESC;"

#run the query
cursor.execute(query)

#fetch the results
results = list(cursor.fetchall())

pp.pprint(results)

[('Art', 'Art', 8253),
 ('Art', 'Painting', 3294),
 ('Art', 'Illustration', 3175),
 ('Art', 'Public Art', 3077),
 ('Art', 'Mixed Media', 2757),
 ('Art', 'Performance Art', 2154),
 ('Art', 'Sculpture', 1810),
 ('Art', 'Digital Art', 1346),
 ('Art', 'Conceptual Art', 1030),
 ('Art', 'Installations', 482),
 ('Art', 'Ceramics', 305),
 ('Art', 'Textiles', 276),
 ('Art', 'Video Art', 194),
 ('Comics', 'Comics', 4996),
 ('Comics', 'Comic Books', 2743),
 ('Comics', 'Graphic Novels', 1864),
 ('Comics', 'Webcomics', 648),
 ('Comics', 'Anthologies', 405),
 ('Comics', 'Events', 163),
 ('Crafts', 'Crafts', 4664),
 ('Crafts', 'DIY', 1173),
 ('Crafts', 'Woodworking', 1167),
 ('Crafts', 'Candles', 429),
 ('Crafts', 'Printing', 238),
 ('Crafts', 'Stationery', 219),
 ('Crafts', 'Knitting', 181),
 ('Crafts', 'Crochet', 162),
 ('Crafts', 'Glass', 138),
 ('Crafts', 'Embroidery', 113),
 ('Crafts', 'Pottery', 100),
 ('Crafts', 'Weaving', 93),
 ('Crafts', 'Quilts', 81),
 ('Crafts', 'Letterpress', 38),
 ('Craf

Return to [Table of Contents](#table)

### 7B. Pandas Method <a name="pandas"></a>

The pandas method presents a more reader-friendly view of the results obtained from the query.

In [232]:
df = pd_sql.read_sql_query(query, connection)

df = (df.groupby(['main_category', 'category'])
        .first()
        .sort_values(['main_category', 'count'], ascending = False))

df.head(25)

count
main_category category                
Theater       Theater             7056
              Plays               1378
              Musical              913
              Festivals            548
              Experimental         373
              Immersive            330
              Spaces               213
              Comedy               101
Technology    Technology          6927
              Apps                6345
              Web                 3906
              Hardware            3662
              Software            3046
              Gadgets             2965
              Wearables           1233
              DIY Electronics      902
              3D Printing          682
              Sound                669
              Robots               572
              Flight               426
              Camera Equipment     416
              Space Exploration    323
              Fabrication Tools    250
              Makerspaces          238
Publishing    Fiction             9168

Return to [Table of Contents](#table)

## 8. Closing the Connection <a name="close"></a>

Do not forget to close the connection once you are finished.

In [233]:
#run this if you are done with psql
cursor.close() 
connection.close()

Return to [Table of Contents](#table)